In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Wed Mar  6 07:45:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
import pandas as pd
import os
import numpy as np
import sys

In [ ]:
os.chdir("/content/drive/MyDrive/FYP_Python_Notebooks/Data_for_School_Computer")
df_outf = pd.read_csv('combined_demand_df_1h.csv')
df_outf = df_outf.drop('time_slot', axis=1)

df_inf = pd.read_csv('combined_supply_df_1h.csv')
df_inf = df_inf.drop('time_slot', axis=1)

if(df_outf.shape[0] != df_inf.shape[0]): # check if dem and inf are same length
    print("outflow and inflow are NOT of same length")
    sys.exit()

In [ ]:
df_outf

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
0,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,7.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,20.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,14.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,5.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,8.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,26.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,19.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,26.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,24.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [ ]:
dt = pd.Timestamp("2021-01-01") + pd.to_timedelta(range(df_inf.shape[0]), "H")
raw_feat_df = pd.DataFrame(columns=['time_idx','datetime','group','target', 'inflow'])

In [ ]:
"""
stack all the raw feature time series of clusters vertically

tsfresh features are then added to these raw features to enlarge the feature set
"""

# inflow means supply, outflow means demand

for c in df_inf.columns:
    # lags_list = zone_significant_lags[c]
    temp_df = pd.DataFrame(columns=['time_idx','datetime','group','target', 'inflow'])
    temp_df['time_idx'] = range(df_inf.shape[0])
    temp_df['datetime'] = dt
    temp_df['target'] = df_outf[c]
    temp_df['group'] = np.repeat(int(c),df_inf.shape[0])
    # temp_df['inflow_lag_0'],temp_df['inflow_lag_1'],temp_df['inflow_lag_2'],temp_df['inflow_lag_3'] = create_raw_features.create_lag_features(df_inf[c],lags_list) # <-- do u need to change lags_list to supply ones -> no dont need
    # temp_df['outflow_lag_0'],temp_df['outflow_lag_1'],temp_df['outflow_lag_2'],temp_df['outflow_lag_3'] = create_raw_features.create_lag_features(df_outf[c],lags_list)

    temp_df['inflow'] = df_inf[c]

    raw_feat_df = pd.concat([raw_feat_df, temp_df],axis=0)

raw_feat_df = raw_feat_df.dropna().reset_index(drop=True)
raw_feat_df

,time_idx,datetime,group,target,inflow
0,0,2021-01-01 00:00:00,4,3.0,8.0
1,1,2021-01-01 01:00:00,4,1.0,13.0
2,2,2021-01-01 02:00:00,4,2.0,9.0
3,3,2021-01-01 03:00:00,4,3.0,5.0
4,4,2021-01-01 04:00:00,4,0.0,2.0
...,...,...,...,...,...
1637653,24808,2023-10-31 16:00:00,263,126.0,153.0
1637654,24809,2023-10-31 17:00:00,263,112.0,135.0
1637655,24810,2023-10-31 18:00:00,263,99.0,119.0
1637656,24811,2023-10-31 19:00:00,263,60.0,119.0


In [ ]:
pip install pyarrow

In [ ]:
os.chdir("/content/drive/MyDrive/FYP_Python_Notebooks/Data_Preprocessing_Notebooks/1_Indiv_Zones")
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data_no_tsfresh.parquet')

In [ ]:
df = pd.read_parquet('combined_dd_ss_raw_feat_data_no_tsfresh.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow
0,0,2021-01-01 00:00:00,4,3.0,8.0
1,1,2021-01-01 01:00:00,4,1.0,13.0
2,2,2021-01-01 02:00:00,4,2.0,9.0
3,3,2021-01-01 03:00:00,4,3.0,5.0
4,4,2021-01-01 04:00:00,4,0.0,2.0
...,...,...,...,...,...
1637653,24808,2023-10-31 16:00:00,263,126.0,153.0
1637654,24809,2023-10-31 17:00:00,263,112.0,135.0
1637655,24810,2023-10-31 18:00:00,263,99.0,119.0
1637656,24811,2023-10-31 19:00:00,263,60.0,119.0
